# Import Packages

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from scipy import stats
from dotenv import load_dotenv   # <- helper for .env files

#Set plot style
plt.style.use("seaborn-v0_8")


# Connect to Neon DB &  Load data

In [ ]:
DB_URL = os.getenv("DATABASE_URL")

# !!!
engine = create_engine(DB_URL)

query =   """
SELECT "Time", "Axis #1", "Axis #2", "Axis #3", "Axis #4",
       "Axis #5", "Axis #6", "Axis #7", "Axis #8"
FROM <table_name>;
"""

df_train = pd.read_sql(query, engine)
# Convert the time column to datetime
df_train["Time"] = pd.to_datetime(df_train["Time"])

# Store the column lists

axis_cols = [f"Axis #{i}" for i in range (1,9)]
time_col = ["Time"]

df_train.head()



# Normalize data (min-max)

In [ ]:
scaler = MinMaxScaler()
df_norm = df_train.copy()
df_norm[axis_cols] = scaler.fit_transform(df_train[axis_cols])

df_norm.head()


# Standardize (Z-scores)


In [ ]:
scaler_std = StandardScaler()

df_std = df_train.copy()
df_std[axis_cols] = scaler_std.fit_transform(df_train[axis_cols])

df_std.head()


# Regression Models (Time > Axis

In [ ]:
df_train["time_seconds"] = (df_train[time_col] - df_train[time_col].min()).dt.total_seconds()

models = {}
residuals = {}

for axis in axis_cols:
    x = df_train[["time_seconds"]].values
    y = df_train[axis].values

    model = LinearRegression().fit(x,y)
    y_pred = model.predict(x)

    models[axis] = model
    residuals[axis] = y - y_pred

    # Plot the regression

    plt.figure(figsize=(8,4))

    plt.scatter(df_train["time_seconds"], y, alpha=0.5, label="Observed")
    plt.plot(df_train["time_seconds"], y_pred, color="red", label="Regression Line")

    plt.title(f"REgression for {axis}")
    plt.xlabel("Time(s)")
    plt.ylabel(axis)
    plt.legend()
    plt.show()


Residual Analysis

In [ ]:
for axis in axis_cols:

    res = residuals[axis]
    plt.figure(figsize=(6,3))
    plt.hist(res, bins=30, alpha=0.7)
    plt.title(f"Residuals Distribution - {axis}")
    plt.xlabel("Residuals")
    plt.ylabel("Frequency")
    plt.show()

Define Min Max

In [ ]:
thresholds = []

for axis in axis_cols:
    res = residuals[axis]

    minC = np.percentile(res, 90) # 90th percinetile deviation sends alert
    maxC = np.percentile(res, 95) # 95th percentile deviation sends error
    T = 5

    thresholds[axis] = {"MinC": minC, "MaxC": maxC, "T": T}

    thresholds

# Detect Alerts  & Errors

In [ ]:

alerts = []

for axis in axis_cols:

    res = residuals[axis]
    MinC = thresholds[axis]["MinC"]
    MaxC = thresholds[axis]["MaxC"]
    T = thresholds[axis]["T"]


    above_min = df_train[res>MinC]
    above_max = df_train[res<MaxC]

    if not above_min.empty:
        alerts.append((axis, "ALERT", above_min[time_col].iloc[0]))
    if not above_max.empty:
        alerts.append((axis,"error", above_max[time_col].iloc[0]))


    pd.DataFrame(alerts, columns = ["Axis", "Type", "Time"])

# Save Results

In [ ]:
# Local save to the results

pd.DataFrame.from_dict(thresholds, orient="index").to_csv("results_thresholds.csv")
pd.DataFrame(alerts, columns = ["Axis", "Type", "Time"]).to_csv("results_detected_events.csv", index=False)

# Explinatnion